# Document Retrival with Title Embedding and IDF on Texts (DR.TEIT)

In this method we used two scoring measure and aggregate them by a convex combination as below:
$$
λ*Similiarity_{Title Embedding} + (1-λ)*Similiarity_{TextIDF}
$$

We used LaBSE model for out embeddings. For computing title embedding similarities we used cosine similarity between query embeddings and each document's title embedding.

For the second part we used character-level (2gram to 8gram). We also trained our TF-IDF transformation matrix on the Multidoc2dial2022 documnets.

## Dataset
### Dataset Description

- **mutldoc2dial_doc.json** contains the documents that are indexed by key `domain` and `doc_id` . Each document instance includes the following,

  - `doc_id`: the ID of a document;
  - `title`: the title of the document;
  - `domain`: the domain of the document;
  - `doc_text`: the text content of the document (without HTML markups);
  - `doc_html_ts`: the document content with HTML markups and the annotated spans that are indicated by `text_id` attribute, which corresponds to `id_sp`.
  - `doc_html_raw`: the document content with HTML markups and without span annotations.
  - `spans`: key-value pairs of all spans in the document, with `id_sp` as key. Each span includes the following,
    - `id_sp`: the id of a  span as noted by `text_id` in  `doc_html_ts`;
    - `start_sp`/  `end_sp`: the start/end position of the text span in `doc_text`;
    - `text_sp`: the text content of the span.
    - `id_sec`: the id of the (sub)section (e.g. `<p>`) or title (`<h2>`) that contains the span.
    - `start_sec` / `end_sec`: the start/end position of the (sub)section in `doc_text`.
    - `text_sec`: the text of the (sub)section.
    - `title`: the title of the (sub)section.
    - `parent_titles`: the parent titles of the `title`.

- **multidoc2dial_dial_train.json** and **multidoc2dial_dial_validation.json**  contain the training and dev split of dialogue data that are indexed by key `domain` . Please note: **For test split, we only include a dummy file in this version.**

  Each dialogue instance includes the following,

  - `dial_id`: the ID of a dialogue;
  - `turns`: a list of dialogue turns. Each turn includes,
    - `turn_id`: the time order of the turn;
    - `role`: either "agent" or "user";READ
    - `da`: dialogue act;
    - `references`: a list of spans with `id_sp` ,  `label` and `doc_id`. `references` is empty if a turn is for indicating previous user query not answerable or irrelevant to the document. **Note** that labels "*precondition*"/"*solution*" are fuzzy annotations that indicate whether a span is for describing a conditional context or a solution.
    - `utterance`: the human-generated utterance based on the dialogue scene.
Downloading the training dataset:

In [1]:
!pip install --upgrade --no-cache-dir gdown

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gdown: filename=gdown-4.4.0-py3-none-any.whl size=14774 sha256=1b7ea770587402ccb50e98736d24b485b2ddd41e04fb2e54c8244ea6f987c135
  Stored in directory: /tmp/pip-ephem-wheel-cache-lna95345/wheels/fb/c3/0e/c4d8ff8bfcb0461afff199471449f642179b74968c15b7a69c
Successfully built gdown
  Attempting uninstall: gdown
    Found existing installation: gdown 4.2.2
    Uninstalling gdown-4.2.2:
      Successfully uninstalled gdown-4.2.2


In [2]:
!gdown --id 1Ln4pU93_ofAkbrz1uibsNABB0QsEaOXw

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1Ln4pU93_ofAkbrz1uibsNABB0QsEaOXw
To: /content/multidoc2dial.zip
100% 6.45M/6.45M [00:00<00:00, 28.6MB/s]


In [3]:
!unzip multidoc2dial.zip

Archive:  multidoc2dial.zip
   creating: multidoc2dial/
  inflating: multidoc2dial/multidoc2dial_dial_validation.json  
  inflating: multidoc2dial/multidoc2dial_dial_train.json  
  inflating: multidoc2dial/multidoc2dial_dial_test.json  
  inflating: multidoc2dial/multidoc2dial_doc.json  
  inflating: multidoc2dial/README.md  


In [4]:
def clean_text(text):
    """
    Clean the given text.

    :param text: input text
    :type text: str
    :return: cleaned string
    """
    return text.strip()

In [5]:
import json
with open('multidoc2dial/multidoc2dial_doc.json', 'r') as f:
    multidoc2dial_doc = json.load(f)

### Extracting titles

In [6]:
titles = []
for doc_idx1 in multidoc2dial_doc['doc_data']:
    for doc_idx2 in multidoc2dial_doc['doc_data'][doc_idx1]:
        titles.append(doc_idx2)
titles

['Benefits Planner: Survivors | Planning For Your Survivors | Social Security Administration#1_0',
 'Benefits Planner: Survivors | Planning For Your Survivors | Social Security Administration#2_0',
 'Benefits Planner: Disability | How You Apply | Social Security Administration#1_0',
 'Benefits Planner: Disability | How You Apply | Social Security Administration#2_0',
 'Learn what documents you will need to get a Social Security Card | Social Security Administration#2_0',
 'Learn what documents you will need to get a Social Security Card | Social Security Administration#3_0',
 'Learn what documents you will need to get a Social Security Card | Social Security Administration#4_0',
 'Learn what documents you will need to get a Social Security Card | Social Security Administration#5_0',
 'Learn what documents you will need to get a Social Security Card | Social Security Administration#6_0',
 'Learn what documents you will need to get a Social Security Card | Social Security Administration#

In [7]:
len(titles)

488

### Extracting document texts

In [8]:
doc_texts_train = []
title_to_domain = {}
for doc_idx1 in multidoc2dial_doc['doc_data']:
    for doc_idx2 in multidoc2dial_doc['doc_data'][doc_idx1]:
        doc_texts_train.append(multidoc2dial_doc['doc_data'][doc_idx1]\
                                          [doc_idx2]['doc_text'].strip())
        title_to_domain[doc_idx2] = doc_idx1
doc_texts_train[10]

"Original Card for a Foreign Born U.S. Citizen Adult \nImportant You must present original documents or copies certified by the agency that issued them. We cannot accept photocopies or notarized copies. All documents must be current not expired. We cannot accept a receipt showing you applied for the document. \n\nWhat original documents do I need? \nCitizenship We can accept only certain documents as proof of U.S. citizenship. These include : U.S. passport ; Certificate of Naturalization N-550/N-570 ; Certificate of Citizenship N-560/N-561 ; Certificate of Report of Birth DS-1350 ; Consular Report of Birth Abroad FS-240, CRBA. Age You must present your foreign birth certificate if you have it or can get it within 10 days. If not , we will consider other documents such as your passport or a document issued by the Department of Homeland Security DHS as evidence of your age. Anyone age 12 or older requesting an original Social Security number must appear in person for an interview. We wil

In [9]:
len(doc_texts_train)

488

## Encoding the sentences
We use the LaBSE which is a Language-agnostic BERT Sentence Encoder (LaBSE) is a BERT-based model trained for sentence embedding for 109 languages. The pre-training process combines masked language modeling with translation language modeling. The model is useful for getting multilingual sentence embeddings and for bi-text retrieval.

In [10]:
!pip install --quiet transformers

     |████████████████████████████████| 4.0 MB 8.3 MB/s 
     |████████████████████████████████| 596 kB 45.2 MB/s 
     |████████████████████████████████| 77 kB 4.7 MB/s 
     |████████████████████████████████| 6.5 MB 49.2 MB/s 
     |████████████████████████████████| 895 kB 69.0 MB/s 


In [11]:
from transformers import AutoTokenizer, AutoModel, AutoConfig
import numpy as np
import torch
from torch.nn.functional import normalize

In [12]:
tokenizer_labse = AutoTokenizer.from_pretrained("setu4993/LaBSE")
model_labse = AutoModel.from_pretrained("setu4993/LaBSE")

Downloading:   0%|          | 0.00/300 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.98M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.18M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/576 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.75G [00:00<?, ?B/s]

### `get_embeddings`
In this method we extract the **pooler output** (Last layer hidden-state of the first token of the sequence (classification token) after further processing through the layers used for the auxiliary pretraining task. E.g. for BERT-family of models, this returns the classification token after processing through a linear layer and a tanh activation function. The linear layer weights are trained from the next sentence prediction (classification) objective during pretraining).

In [13]:
def get_embeddings(sentece):
    """
    Return embeddings based on encoder model

    :param sentence: input sentence(s)
    :type sentence: str or list of strs
    :return: embeddings
    """
    tokenized = tokenizer_labse(sentece,
                                return_tensors="pt",
                                padding=True)
    with torch.no_grad():
        embeddings = model_labse(**tokenized)
    
    return np.squeeze(np.array(embeddings.pooler_output))

### Title embedding

In [14]:
title_embeddings = []
progress = 0
TRAIN_SIZE = len(titles)
for title in titles:
    title_embeddings.append(get_embeddings(title))
    progress += 1
    if progress % 50 == 0:
        print('Progress Percent = {}%'.format(100 * progress / TRAIN_SIZE))

Progress Percent = 10.245901639344263%
Progress Percent = 20.491803278688526%
Progress Percent = 30.737704918032787%
Progress Percent = 40.98360655737705%
Progress Percent = 51.22950819672131%
Progress Percent = 61.47540983606557%
Progress Percent = 71.72131147540983%
Progress Percent = 81.9672131147541%
Progress Percent = 92.21311475409836%


In [15]:
with open('doc_title_LaBSE_Embedding.npy', 'wb') as f:
    np.save(f, np.array(title_embeddings))

In [16]:
title_to_embeddings = {}
progress = 0
TRAIN_SIZE = len(titles)
for title in titles:
    title_to_embeddings[title] = get_embeddings(title)
    progress += 1
    if progress % 50 == 0:
        print('Progress Percent = {}%'.format(100 * progress / TRAIN_SIZE))

Progress Percent = 10.245901639344263%
Progress Percent = 20.491803278688526%
Progress Percent = 30.737704918032787%
Progress Percent = 40.98360655737705%
Progress Percent = 51.22950819672131%
Progress Percent = 61.47540983606557%
Progress Percent = 71.72131147540983%
Progress Percent = 81.9672131147541%
Progress Percent = 92.21311475409836%


In [17]:
import pickle
with open('title_to_embeddings.pkl', 'wb') as f:
    pickle.dump(title_to_embeddings, f)

## Calculating the IDF for each token

In [18]:
words = set()
doc_texts_train_tokenized = []
for doc in doc_texts_train:
    tokenized_doc = [s.lower() for s in tokenizer_labse.tokenize(doc)]
    doc_texts_train_tokenized.append(tokenized_doc) 
    words = set(tokenized_doc).union(words)
len(words)

8446

In [19]:
words2IDF = {}
N_doc = len(doc_texts_train)
for i, word in enumerate(words):
    n_word = 0
    for doc in doc_texts_train_tokenized:
        if word in doc:
            n_word += 1
    words2IDF[word] = np.log(N_doc / (n_word + 1))
    if i % 1000 == 0:
        print(word, words2IDF[word])

granted 3.482265204750937
borrow 2.526753759723501
402 4.580877493419047
npr 5.0917031171850375
bonus 4.804021044733257
##bile 5.497168225293202
19 2.5527292461267614
institutional 4.580877493419047
extended 2.7245795030534206


In [20]:
len(words2IDF)

8446

In [21]:
def calc_idf_score(sentence):
    """
    Calculate the mean idf score for given sentence.

    :param sentence: input sentence
    :type sentence: str
    :return: mean idf score of sentence token
    """
    tokenzied_sentence = [s.lower() for s in tokenizer_labse.tokenize(sentence)]
    score = 0
    for token in tokenzied_sentence:
        if token in words2IDF:
            score += words2IDF[token]
        else:
            score += np.log(N_doc)
    return score / len(tokenzied_sentence)

### Saving the IDF values dictionary

In [22]:
import pickle
with open('IDFs.pkl', 'wb') as f:
    pickle.dump(words2IDF, f)

## Methods

### IDF only - Vanilla

This is the first method which we used for document retriver. In this method we just used similarity between query embeddign and document title embedding. We also used IDF scores as a factor for history queries by which we can pay more attention to the more informative query.

In [23]:
def predict_labelwise_doc_at_history(queries, k=1):
    """
    Predict which document is matched to the given query.

    :param queries: input queries in time reversed order (latest first)
    :type queries: str (or list of strs)
    :param k: number of returning docs
    :type k: int 
    :return: return the document names and accuracies
    """
    similarities = np.array(list(map(lambda x: 0.0, title_embeddings)))
    coef_sum = 0
    for query in queries:
        query_embd = get_embeddings(query)
        query_sim = list(map(lambda x: np.dot(x, query_embd) /
                            (np.linalg.norm(query_embd) * np.linalg.norm(x)),
                            title_embeddings))
        query_sim = np.array(query_sim)

        coef = calc_idf_score(query)
        coef_sum += coef
        similarities += coef * query_sim

    similarities = similarities / coef_sum
    best_k_idx = similarities.argsort()[::-1][:k]
    predictions = list(map(lambda x: titles[x], best_k_idx))
    accuracy = similarities[best_k_idx]
    return accuracy, predictions

### IDF ordered

In the last method there wasn't any difference between query and histories - all sentences would be treat same regardless of their queried time. From now we will use a reweighting (multiplying query's score to $2^{-i}$ when $i$ is the index of query) system which favour last query more.

In [24]:
def predict_labelwise_doc_at_history_ordered(queries, k=1):
    """
    Predict which document is matched to the given query.

    :param queries: input queries in time reversed order (latest first)
    :type queries: str (or list of strs)
    :param k: number of returning docs
    :type k: int 
    :return: return the document names and accuracies
    """
    similarities = np.array(list(map(lambda x: 0.0, title_embeddings)))
    coef_sum = 0
    for i, query in enumerate(queries):
        query_embd = get_embeddings(query)
        query_sim = list(map(lambda x: np.dot(x, query_embd) /
                            (np.linalg.norm(query_embd) * np.linalg.norm(x)),
                            title_embeddings))
        query_sim = np.array(query_sim)

        coef = 2**(-i) * calc_idf_score(query)
        coef_sum += coef
        similarities += coef * query_sim

    similarities = similarities / coef_sum
    best_k_idx = similarities.argsort()[::-1][:k]
    predictions = list(map(lambda x: titles[x], best_k_idx))
    accuracy = similarities[best_k_idx]
    return accuracy, predictions

### IDF ordered - softmaxed
In this method we changed a little bit. Instead of using coefitionts barely we apply the softmax function favouring the maximum score more. But results were not good. 

In [25]:
def predict_labelwise_doc_at_history_ordered_softmaxed(queries, k=1):
    """
    Predict which document is matched to the given query.

    :param queries: input queries in time reversed order (latest first)
    :type queries: str (or list of strs)
    :param k: number of returning docs
    :type k: int 
    :return: return the document names and accuracies
    """
    similarities = np.array(list(map(lambda x: 0.0, title_embeddings)))
    coefs = []
    sims = []
    for i, query in enumerate(queries):
        query_embd = get_embeddings(query)
        query_sim = list(map(lambda x: np.dot(x, query_embd) /
                            (np.linalg.norm(query_embd) * np.linalg.norm(x)),
                            title_embeddings))
        sims.append(np.array(query_sim))
        coefs.append(2**(-i) * calc_idf_score(query))
    
    # Softmax:
    coefs = np.array(list(map(lambda x: np.exp(-x), coefs)))
    coefs /= coefs.sum()
    coefs = list(coefs)

    for coef, sim in zip(coefs, sims):
        similarities += coef * sim
    best_k_idx = similarities.argsort()[::-1][:k]
    predictions = list(map(lambda x: titles[x], best_k_idx))
    accuracy = similarities[best_k_idx]
    return accuracy, predictions

### IDF + self attention (cosine sim)

In this method we changed the reweighting method from power-order ($2^{-i}$) to a feed-forward self-attention mechanism. Here we just simply use cosine similarity between each history and query as its coefficient. As it's showing how far is it from the main query.

In [26]:
def predict_labelwise_doc_at_history_selfatt(queries, k=1):
    """
    Predict which document is matched to the given query.

    :param queries: input queries in time reversed order (latest first)
    :type queries: str (or list of strs)
    :param k: number of returning docs
    :type k: int 
    :return: return the document names and accuracies
    """
    similarities = np.array(list(map(lambda x: 0.0, title_embeddings)))
    coef_sum = 0
    query0_embd = get_embeddings(queries[0])
    for query in queries:
        query_embd = get_embeddings(query)
        query_sim = list(map(lambda x: np.dot(x, query_embd) /
                            (np.linalg.norm(query_embd) * np.linalg.norm(x)),
                            title_embeddings))
        query_sim = np.array(query_sim)

        coef = calc_idf_score(query) * np.dot(query0_embd, query_embd) / (np.linalg.norm(query_embd) * np.linalg.norm(query0_embd))
        coef_sum += coef
        similarities += coef * query_sim

    similarities = similarities / coef_sum
    best_k_idx = similarities.argsort()[::-1][:k]
    predictions = list(map(lambda x: titles[x], best_k_idx))
    accuracy = similarities[best_k_idx]
    return accuracy, predictions

### DR. TEIT*

In this method we used two scoring measure and aggregate them by a convex combination as below:
$$
λ*Similiarity_{Title Embedding} + (1-λ)*Similiarity_{TextIDF}
$$

We used LaBSE model for out embeddings. For computing title embedding similarities we used cosine similarity between query embeddings and each document's title embedding.

For the second part we used character-level (2gram to 8gram). We also trained our TF-IDF transformation matrix on the Multidoc2dial2022 documnets.

**NOTE: In `predict_DR_TEIT` you may see a diffrent notation (`alpha`) but they are the same.**

#### TF-IDF Transformation Matrix Fitting

In [28]:
doc_texts_train = []
for doc_idx1 in multidoc2dial_doc['doc_data']:
    for doc_idx2 in multidoc2dial_doc['doc_data'][doc_idx1]:
        doc_texts_train.append(multidoc2dial_doc['doc_data'][doc_idx1]\
                                          [doc_idx2]['doc_text'].strip())
doc_texts_train[10]

"Original Card for a Foreign Born U.S. Citizen Adult \nImportant You must present original documents or copies certified by the agency that issued them. We cannot accept photocopies or notarized copies. All documents must be current not expired. We cannot accept a receipt showing you applied for the document. \n\nWhat original documents do I need? \nCitizenship We can accept only certain documents as proof of U.S. citizenship. These include : U.S. passport ; Certificate of Naturalization N-550/N-570 ; Certificate of Citizenship N-560/N-561 ; Certificate of Report of Birth DS-1350 ; Consular Report of Birth Abroad FS-240, CRBA. Age You must present your foreign birth certificate if you have it or can get it within 10 days. If not , we will consider other documents such as your passport or a document issued by the Department of Homeland Security DHS as evidence of your age. Anyone age 12 or older requesting an original Social Security number must appear in person for an interview. We wil

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfVectorizer = TfidfVectorizer(strip_accents=None,
                                 analyzer='char',
                                 ngram_range=(2, 8),
                                 norm='l2',
                                 use_idf=True,
                                 smooth_idf=True)
tfidf_wm = tfidfVectorizer.fit_transform(doc_texts_train)

In [30]:
import pickle
with open('tfidfVectorizer.pkl', 'wb') as f:
    pickle.dump(tfidfVectorizer, f)

with open('tfidf_wm.pkl', 'wb') as f:
    pickle.dump(tfidf_wm, f)

#### DR. TEIT

In [32]:
def predict_DR_TEIT(queries, k=1, alpha=10):
    """
    Predict which document is matched to the given query.

    :param queries: input queries in time reversed order (latest first)
    :type queries: str (or list of strs)
    :param k: number of returning docs
    :type k: int 
    :return: return the document names and accuracies
    """

    idf_score = np.array(list(map(lambda x: 0.0, title_embeddings)))
    tfidf_score = np.array(list(map(lambda x: 0.0, title_embeddings)))
    coef_sum = 0
    for i, query in enumerate(queries):
        query_embd = get_embeddings(query)
        query_sim = list(map(lambda x: np.dot(x, query_embd) /
                            (np.linalg.norm(query_embd) * np.linalg.norm(x)),
                            title_embeddings))
        query_sim = np.array(query_sim)
        coef = 2**(-i) * calc_idf_score(query)
        coef_sum += coef

        idf_score += coef * query_sim
        tfidf_score += coef * np.squeeze(np.asarray(tfidf_wm @ tfidfVectorizer.transform([query]).todense().T))

    scores = (idf_score + alpha * tfidf_score) / coef_sum
    best_k_idx = scores.argsort()[::-1][:k]
    scores = scores[best_k_idx]
    predictions = list(map(lambda x: titles[x], best_k_idx))
    return (scores, predictions)

## Test
In the test dataset we just picked ones with **user** turn.

In [33]:
test_queries = ["I'm looking for information regarding benefits planning, can you help me?",
                "I want to know about the benefits plan for survivors, can you give me more information about this?",
                "What are Social Security credits?"]
test_labels = ["Benefits Planner: Survivors | Planning For Your Survivors | Social Security Administration#1_0",
               "Benefits Planner: Survivors | Planning For Your Survivors | Social Security Administration#1_0",
               "Benefits Planner: Survivors | Planning For Your Survivors | Social Security Administration#1_0"]

In [34]:
import json
with open('multidoc2dial/multidoc2dial_dial_train.json', 'r') as f:
    multidoc2dial_dial_train = json.load(f)

In [35]:
doc_sentence_test = []
doc_label_test = []
for doc_idx1 in multidoc2dial_dial_train['dial_data']:
    for dial in multidoc2dial_dial_train['dial_data'][doc_idx1]:
        for turns in dial['turns']:
            if turns['role'] == "user":
                doc_sentence_test.append(turns['utterance'])
                doc_label_test.append(turns['references'][0]['doc_id'])

In [36]:
TEST_SIZE = len(doc_sentence_test)
TEST_SIZE

23399

In [46]:
TEST_SIZE = TEST_SIZE // 20   #   For making it faster

### IDF only - Vanilla

In [47]:
accs, preds = predict_labelwise_doc_at_history([test_queries[2],
                                               test_queries[1],
                                               test_queries[0]],
                                               k=5)
print(accs)
print(preds)
print('-' * 20)

[0.37842299 0.37675699 0.37660415 0.37555711 0.37375754]
['Benefits Planner: Survivors | How You Apply | Social Security Administration#1_0', 'Benefits Planner: Survivors | How You Apply | Social Security Administration#2_0', 'Learn what documents you will need to get a Social Security Card | Social Security Administration#6_0_1_2_3', 'Learn what documents you will need to get a Social Security Card | Social Security Administration#6_0_1_2', 'How To Apply For Social Security Disability Benefits#1_0']
--------------------


In [48]:
prec_at_500 = 0
prec_at_100 = 0
prec_at_50 = 0
prec_at_10 = 0
prec_at_5 = 0
prec_at_1 = 0
sample_till_now = 0
ranks = []
for i in range(2, TEST_SIZE):
    act_doc = doc_label_test[i]
    accs, preds = predict_labelwise_doc_at_history([doc_sentence_test[i],
                                                   doc_sentence_test[i-1],
                                                   doc_sentence_test[i-2]],
                                                   k=500)
    ranks.append(1 / (preds.index(act_doc) + 1))
    if act_doc == preds[0]:
        prec_at_1 += 1
    if act_doc in preds[:5]:
        prec_at_5 += 1
    if act_doc in preds[:10]:
        prec_at_10 += 1
    if act_doc in preds[:50]:
        prec_at_50 += 1
    if act_doc in preds[:100]:
        prec_at_100 += 1
    if act_doc in preds[:500]:
        prec_at_500 += 1
    sample_till_now += 1
    if sample_till_now % 100 == 0:
        print("MRR: mean={}, var={}".format(np.array(ranks).mean(), np.array(ranks).var()))
        print("Prec@(1) = {} | Prec@(5) = {} | Prec@(10) = {} | Prec@(50) = {} | Prec@(100) = {} | Prec@(500) = {} | NUMBER_OF_SAMPLES = {}".\
              format(prec_at_1 / sample_till_now, prec_at_5 / sample_till_now,
                     prec_at_10 / sample_till_now, prec_at_50 / sample_till_now,
                     prec_at_100 / sample_till_now, prec_at_500 / sample_till_now,
                     sample_till_now))

MRR: mean=0.11905440352084566, var=0.05986443862836907
Prec@(1) = 0.06 | Prec@(5) = 0.15 | Prec@(10) = 0.23 | Prec@(50) = 0.47 | Prec@(100) = 0.69 | Prec@(500) = 1.0 | NUMBER_OF_SAMPLES = 100
MRR: mean=0.11182382229775921, var=0.04994596429538301
Prec@(1) = 0.045 | Prec@(5) = 0.155 | Prec@(10) = 0.25 | Prec@(50) = 0.5 | Prec@(100) = 0.715 | Prec@(500) = 1.0 | NUMBER_OF_SAMPLES = 200
MRR: mean=0.1327195627539497, var=0.06172267295242753
Prec@(1) = 0.06 | Prec@(5) = 0.19 | Prec@(10) = 0.29 | Prec@(50) = 0.5366666666666666 | Prec@(100) = 0.76 | Prec@(500) = 1.0 | NUMBER_OF_SAMPLES = 300
MRR: mean=0.1627914543093306, var=0.07950550918949202
Prec@(1) = 0.085 | Prec@(5) = 0.2325 | Prec@(10) = 0.335 | Prec@(50) = 0.59 | Prec@(100) = 0.795 | Prec@(500) = 1.0 | NUMBER_OF_SAMPLES = 400
MRR: mean=0.18872528867358432, var=0.09459400049203383
Prec@(1) = 0.108 | Prec@(5) = 0.266 | Prec@(10) = 0.364 | Prec@(50) = 0.622 | Prec@(100) = 0.816 | Prec@(500) = 1.0 | NUMBER_OF_SAMPLES = 500
MRR: mean=0.1826

### IDF ordered

In [49]:
accs, preds = predict_labelwise_doc_at_history_ordered([test_queries[2],
                                                        test_queries[1],
                                                        test_queries[0]],
                                                        k=5)
print(accs)
print(preds)
print('-' * 20)

[0.41813262 0.41414311 0.413184   0.41088731 0.40784173]
['Learn what documents you will need to get a Social Security Card | Social Security Administration#6_0_1_2_3', 'Learn what documents you will need to get a Social Security Card | Social Security Administration#6_0_1_2', 'Learn what documents you will need to get a Social Security Card | Social Security Administration#7_0_1_2', 'Learn what documents you will need to get a Social Security Card | Social Security Administration#6_0_1_2_3_4', 'Learn what documents you will need to get a Social Security Card | Social Security Administration#12_0_1_2']
--------------------


In [50]:
prec_at_500 = 0
prec_at_100 = 0
prec_at_50 = 0
prec_at_10 = 0
prec_at_5 = 0
prec_at_1 = 0
sample_till_now = 0
ranks = []
for i in range(2, TEST_SIZE):
    act_doc = doc_label_test[i]
    accs, preds = predict_labelwise_doc_at_history_ordered([doc_sentence_test[i],
                                                            doc_sentence_test[i-1],
                                                            doc_sentence_test[i-2]],
                                                            k=500)
    ranks.append(1 / (preds.index(act_doc) + 1))
    if act_doc == preds[0]:
        prec_at_1 += 1
    if act_doc in preds[:5]:
        prec_at_5 += 1
    if act_doc in preds[:10]:
        prec_at_10 += 1
    if act_doc in preds[:50]:
        prec_at_50 += 1
    if act_doc in preds[:100]:
        prec_at_100 += 1
    if act_doc in preds[:500]:
        prec_at_500 += 1
    sample_till_now += 1
    if sample_till_now % 100 == 0:
        print("MRR: mean={}, var={}".format(np.array(ranks).mean(), np.array(ranks).var()))
        print("Prec@(1) = {} | Prec@(5) = {} | Prec@(10) = {} | Prec@(50) = {} | Prec@(100) = {} | Prec@(500) = {} | NUMBER_OF_SAMPLES = {}".\
              format(prec_at_1 / sample_till_now, prec_at_5 / sample_till_now,
                     prec_at_10 / sample_till_now, prec_at_50 / sample_till_now,
                     prec_at_100 / sample_till_now, prec_at_500 / sample_till_now,
                     sample_till_now))

MRR: mean=0.144511049798484, var=0.08118642286253466
Prec@(1) = 0.09 | Prec@(5) = 0.16 | Prec@(10) = 0.28 | Prec@(50) = 0.53 | Prec@(100) = 0.72 | Prec@(500) = 1.0 | NUMBER_OF_SAMPLES = 100
MRR: mean=0.15031672380406347, var=0.08741299578833683
Prec@(1) = 0.1 | Prec@(5) = 0.165 | Prec@(10) = 0.275 | Prec@(50) = 0.545 | Prec@(100) = 0.76 | Prec@(500) = 1.0 | NUMBER_OF_SAMPLES = 200
MRR: mean=0.17559390346365905, var=0.09533169069906142
Prec@(1) = 0.11 | Prec@(5) = 0.23 | Prec@(10) = 0.3333333333333333 | Prec@(50) = 0.5633333333333334 | Prec@(100) = 0.7833333333333333 | Prec@(500) = 1.0 | NUMBER_OF_SAMPLES = 300
MRR: mean=0.1914336875137787, var=0.09923098973353595
Prec@(1) = 0.115 | Prec@(5) = 0.2625 | Prec@(10) = 0.3625 | Prec@(50) = 0.605 | Prec@(100) = 0.8075 | Prec@(500) = 1.0 | NUMBER_OF_SAMPLES = 400
MRR: mean=0.21154299103948063, var=0.10798957880277361
Prec@(1) = 0.13 | Prec@(5) = 0.282 | Prec@(10) = 0.398 | Prec@(50) = 0.646 | Prec@(100) = 0.83 | Prec@(500) = 1.0 | NUMBER_OF_SA

### IDF ordered - softmaxed

In [51]:
accs, preds = predict_labelwise_doc_at_history_ordered_softmaxed([test_queries[2],
                                                        test_queries[1],
                                                        test_queries[0]],
                                                        k=5)
print(accs)
print(preds)
print('-' * 20)

[0.37505364 0.37495323 0.36257429 0.36173653 0.36112785]
['Benefits Planner: Survivors | How You Apply | Social Security Administration#1_0', 'Benefits Planner: Survivors | How You Apply | Social Security Administration#2_0', 'Benefits Planner: Disability | How You Apply | Social Security Administration#2_0', 'Learn About Retirement Benefits | SSA#1_0', 'Benefits Planner: Disability | How You Apply | Social Security Administration#1_0']
--------------------


In [52]:
prec_at_500 = 0
prec_at_100 = 0
prec_at_50 = 0
prec_at_10 = 0
prec_at_5 = 0
prec_at_1 = 0
sample_till_now = 0
ranks = []
for i in range(2, TEST_SIZE):
    act_doc = doc_label_test[i]
    accs, preds = predict_labelwise_doc_at_history_ordered_softmaxed([doc_sentence_test[i],
                                                            doc_sentence_test[i-1],
                                                            doc_sentence_test[i-2]],
                                                            k=500)
    ranks.append(1 / (preds.index(act_doc) + 1))
    if act_doc == preds[0]:
        prec_at_1 += 1
    if act_doc in preds[:5]:
        prec_at_5 += 1
    if act_doc in preds[:10]:
        prec_at_10 += 1
    if act_doc in preds[:50]:
        prec_at_50 += 1
    if act_doc in preds[:100]:
        prec_at_100 += 1
    if act_doc in preds[:500]:
        prec_at_500 += 1
    sample_till_now += 1
    if sample_till_now % 100 == 0:
        print("MRR: mean={}, var={}".format(np.array(ranks).mean(), np.array(ranks).var()))
        print("Prec@(1) = {} | Prec@(5) = {} | Prec@(10) = {} | Prec@(50) = {} | Prec@(100) = {} | Prec@(500) = {} | NUMBER_OF_SAMPLES = {}".\
              format(prec_at_1 / sample_till_now, prec_at_5 / sample_till_now,
                     prec_at_10 / sample_till_now, prec_at_50 / sample_till_now,
                     prec_at_100 / sample_till_now, prec_at_500 / sample_till_now,
                     sample_till_now))

MRR: mean=0.09269418454220091, var=0.041466296907041615
Prec@(1) = 0.04 | Prec@(5) = 0.12 | Prec@(10) = 0.24 | Prec@(50) = 0.44 | Prec@(100) = 0.6 | Prec@(500) = 1.0 | NUMBER_OF_SAMPLES = 100
MRR: mean=0.090842020703304, var=0.042118003791540175
Prec@(1) = 0.04 | Prec@(5) = 0.11 | Prec@(10) = 0.21 | Prec@(50) = 0.44 | Prec@(100) = 0.62 | Prec@(500) = 1.0 | NUMBER_OF_SAMPLES = 200
MRR: mean=0.11415570292589938, var=0.05874265241044486
Prec@(1) = 0.06 | Prec@(5) = 0.14 | Prec@(10) = 0.24666666666666667 | Prec@(50) = 0.4633333333333333 | Prec@(100) = 0.6766666666666666 | Prec@(500) = 1.0 | NUMBER_OF_SAMPLES = 300
MRR: mean=0.14876841730908613, var=0.08039533161912413
Prec@(1) = 0.0875 | Prec@(5) = 0.19 | Prec@(10) = 0.29 | Prec@(50) = 0.5175 | Prec@(100) = 0.7275 | Prec@(500) = 1.0 | NUMBER_OF_SAMPLES = 400
MRR: mean=0.16919351545310174, var=0.09242517954525063
Prec@(1) = 0.104 | Prec@(5) = 0.22 | Prec@(10) = 0.306 | Prec@(50) = 0.546 | Prec@(100) = 0.742 | Prec@(500) = 1.0 | NUMBER_OF_SA

### IDF + self attention (cosine sim)

In [53]:
accs, preds = predict_labelwise_doc_at_history_selfatt([test_queries[2],
                                                        test_queries[1],
                                                        test_queries[0]],
                                                        k=5)
print(accs)
print(preds)
print('-' * 20)

[0.4257137  0.42203476 0.42132395 0.41824857 0.4159181 ]
['Learn what documents you will need to get a Social Security Card | Social Security Administration#6_0_1_2_3', 'Learn what documents you will need to get a Social Security Card | Social Security Administration#6_0_1_2', 'Learn what documents you will need to get a Social Security Card | Social Security Administration#7_0_1_2', 'Learn what documents you will need to get a Social Security Card | Social Security Administration#6_0_1_2_3_4', 'Learn what documents you will need to get a Social Security Card | Social Security Administration#12_0_1_2']
--------------------


In [54]:
prec_at_500 = 0
prec_at_100 = 0
prec_at_50 = 0
prec_at_10 = 0
prec_at_5 = 0
prec_at_1 = 0
sample_till_now = 0
ranks = []
for i in range(2, TEST_SIZE):
    act_doc = doc_label_test[i]
    accs, preds = predict_labelwise_doc_at_history_selfatt([doc_sentence_test[i],
                                                            doc_sentence_test[i-1],
                                                            doc_sentence_test[i-2]],
                                                            k=500)
    ranks.append(1 / (preds.index(act_doc) + 1))
    if act_doc == preds[0]:
        prec_at_1 += 1
    if act_doc in preds[:5]:
        prec_at_5 += 1
    if act_doc in preds[:10]:
        prec_at_10 += 1
    if act_doc in preds[:50]:
        prec_at_50 += 1
    if act_doc in preds[:100]:
        prec_at_100 += 1
    if act_doc in preds[:500]:
        prec_at_500 += 1
    sample_till_now += 1
    if sample_till_now % 100 == 0:
        print("MRR: mean={}, var={}".format(np.array(ranks).mean(), np.array(ranks).var()))
        print("Prec@(1) = {} | Prec@(5) = {} | Prec@(10) = {} | Prec@(50) = {} | Prec@(100) = {} | Prec@(500) = {} | NUMBER_OF_SAMPLES = {}".\
              format(prec_at_1 / sample_till_now, prec_at_5 / sample_till_now,
                     prec_at_10 / sample_till_now, prec_at_50 / sample_till_now,
                     prec_at_100 / sample_till_now, prec_at_500 / sample_till_now,
                     sample_till_now))

MRR: mean=0.13712179858232124, var=0.06804061197193524
Prec@(1) = 0.07 | Prec@(5) = 0.17 | Prec@(10) = 0.31 | Prec@(50) = 0.49 | Prec@(100) = 0.72 | Prec@(500) = 1.0 | NUMBER_OF_SAMPLES = 100
MRR: mean=0.1407645002818882, var=0.07746681079191914
Prec@(1) = 0.085 | Prec@(5) = 0.165 | Prec@(10) = 0.28 | Prec@(50) = 0.505 | Prec@(100) = 0.755 | Prec@(500) = 1.0 | NUMBER_OF_SAMPLES = 200
MRR: mean=0.16801966978901922, var=0.08910630754713314
Prec@(1) = 0.1 | Prec@(5) = 0.22333333333333333 | Prec@(10) = 0.33 | Prec@(50) = 0.5266666666666666 | Prec@(100) = 0.7733333333333333 | Prec@(500) = 1.0 | NUMBER_OF_SAMPLES = 300
MRR: mean=0.17874436936684396, var=0.09063010654135949
Prec@(1) = 0.1025 | Prec@(5) = 0.2475 | Prec@(10) = 0.355 | Prec@(50) = 0.5775 | Prec@(100) = 0.795 | Prec@(500) = 1.0 | NUMBER_OF_SAMPLES = 400
MRR: mean=0.20205575728498712, var=0.10276163124862168
Prec@(1) = 0.122 | Prec@(5) = 0.274 | Prec@(10) = 0.388 | Prec@(50) = 0.618 | Prec@(100) = 0.824 | Prec@(500) = 1.0 | NUMBER

### DR.TEIT

In [57]:
prec_at_500 = 0
prec_at_100 = 0
prec_at_50 = 0
prec_at_10 = 0
prec_at_5 = 0
prec_at_1 = 0
sample_till_now = 0
ranks = []
for i in range(2, TEST_SIZE):
    act_doc = doc_label_test[i]
    accs, preds = predict_DR_TEIT([doc_sentence_test[i],
                                   doc_sentence_test[i-1],
                                   doc_sentence_test[i-2]],
                                   k=500,
                                   alpha=10)
    ranks.append(1 / (preds.index(act_doc) + 1))
    if act_doc == preds[0]:
        prec_at_1 += 1
    if act_doc in preds[:5]:
        prec_at_5 += 1
    if act_doc in preds[:10]:
        prec_at_10 += 1
    if act_doc in preds[:50]:
        prec_at_50 += 1
    if act_doc in preds[:100]:
        prec_at_100 += 1
    if act_doc in preds[:500]:
        prec_at_500 += 1
    sample_till_now += 1
    if sample_till_now % 100 == 0:
        print("MRR: mean={}, var={}".format(np.array(ranks).mean(), np.array(ranks).var()))
        print("Prec@(1) = {} | Prec@(5) = {} | Prec@(10) = {} | Prec@(50) = {} | Prec@(100) = {} | Prec@(500) = {} | NUMBER_OF_SAMPLES = {}".\
              format(prec_at_1 / sample_till_now, prec_at_5 / sample_till_now,
                     prec_at_10 / sample_till_now, prec_at_50 / sample_till_now,
                     prec_at_100 / sample_till_now, prec_at_500 / sample_till_now,
                     sample_till_now))

MRR: mean=0.6718437625222385, var=0.14197122368318632
Prec@(1) = 0.54 | Prec@(5) = 0.81 | Prec@(10) = 0.9 | Prec@(50) = 0.99 | Prec@(100) = 0.99 | Prec@(500) = 1.0 | NUMBER_OF_SAMPLES = 100
MRR: mean=0.7621047653992961, var=0.12164531618233917
Prec@(1) = 0.66 | Prec@(5) = 0.875 | Prec@(10) = 0.925 | Prec@(50) = 0.98 | Prec@(100) = 0.99 | Prec@(500) = 1.0 | NUMBER_OF_SAMPLES = 200
MRR: mean=0.7513653919166343, var=0.12320002480578507
Prec@(1) = 0.64 | Prec@(5) = 0.8733333333333333 | Prec@(10) = 0.9133333333333333 | Prec@(50) = 0.9766666666666667 | Prec@(100) = 0.9866666666666667 | Prec@(500) = 1.0 | NUMBER_OF_SAMPLES = 300
MRR: mean=0.7549416852811675, var=0.12332384603430972
Prec@(1) = 0.6475 | Prec@(5) = 0.8775 | Prec@(10) = 0.9175 | Prec@(50) = 0.975 | Prec@(100) = 0.9825 | Prec@(500) = 1.0 | NUMBER_OF_SAMPLES = 400
MRR: mean=0.7485866171646938, var=0.12128808971943039
Prec@(1) = 0.632 | Prec@(5) = 0.89 | Prec@(10) = 0.926 | Prec@(50) = 0.974 | Prec@(100) = 0.986 | Prec@(500) = 1.0 |

## Results

At last we have resutls as follows:


| Method | @1 | @5 | @10 | @50 | @100 | MRR (mean, var) |
|:------:|:------:|:------:|:-------:|:-------:|:--------:|:---:|
| IDF - vanilla | 13% | 30% | 39% | 64% | 83% | (0.22, 0.11) |
| IDF - power-order | 15% | 31% | 41% | 65% | 83% | (0.23, 0.12) |
| IDF - power-order (softmax) | 10.7% | 23% | 31% | 57.6% | 78% | (0.18, 0.09) |
| IDF - self-attention | 13.9% | 29% | 38% | 62% | 82% | (0.22, 0.11) |
| **DR. TEIT** | **61.6%** | **86%** | **91%** | **96%** | **98%** | **(0.72, 0.13)** |

It shows that title informations were not enough for document retrieval.

# drafts

In [58]:
tfidf_wm.shape

(488, 1047632)

In [59]:
answers = tfidfVectorizer.transform(["Original Card for a Foreign Born U.S. Citizen Adult",
                                     "Hello world from far beyound!"]).todense()
query = tfidfVectorizer.transform(["Hello!"]).todense()

In [60]:
print(answers.shape, query.shape)

(2, 1047632) (1, 1047632)


In [61]:
import numpy as np
answers_sim = np.squeeze(np.asarray(tfidf_wm @ answers.T))
query_sim = np.squeeze(np.asarray(tfidf_wm @ query.T))

In [62]:
print(answers_sim.shape, query_sim.shape)

(488, 2) (488,)


In [63]:
list(map(lambda x: np.dot(x, query_sim) /
        (np.linalg.norm(query_sim) * np.linalg.norm(x)),
        answers_sim.T))

[0.7506911990367025, 0.934114716518692]